In [1]:
import numpy as np
import os
from glob import glob
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import cv2
import matplotlib.pyplot as plt
from math import floor
import pickle
import time
from natsort import natsorted

/misc/me/pratikm/virtualEnv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
tf.enable_eager_execution()

## Global variables

In [3]:
DIR_DATA = 'data_multiview'
DIR_INPUT = os.path.join(DIR_DATA, 'annotated_frames')
DIR_OUTPUT = os.path.join(DIR_DATA, 'bounding_boxes')

OBJECT_LABELS = {
    'hand': (0, 'hand')
}
NUM_OBJECTS = 1
DIM_OUTPUT_PER_GRID_PER_ANCHOR = 5 + NUM_OBJECTS

# Reference: https://github.com/pjreddie/darknet/blob/master/cfg/yolo-voc.cfg#L242 
GRID_H, GRID_W = 13, 13 
GRID_SIZE = 416//GRID_H
ANCHORS = np.array(
    [
        [0.09112895, 0.06958421],
        [0.21102316, 0.16803947],
        [0.42625895, 0.26609842],
        [0.25476474, 0.49848   ],
        [0.52668947, 0.59138947]
    ]
)
NUM_ANCHORS = ANCHORS.shape[0]
ANCHORS *= np.array([GRID_H, GRID_W])  # map from [0,1] space to [0,19] space
IMG_OUT_H, IMG_OUT_W = GRID_H * GRID_SIZE, GRID_W * GRID_SIZE 

DIR_TFRECORDS = DIR_DATA + '/' + 'data_hand_multiview_tfrecords'
NUM_EXAMPLES_PER_TFRECORD = 500

## Process data

In [4]:
def pose2corner(pose):
    # pose.shape = [21, 2] in uv (image) space
    xy_min = np.min(pose, axis=0)
    xy_max = np.max(pose, axis=0)
    
    return np.array([xy_min[1], xy_min[0], xy_max[1], xy_max[0]], dtype=np.float32)


def normalize_data(img, target):
    # resize input
    img_in_h = img.shape[0]
    img_in_w = img.shape[1]
    img = cv2.resize(img, (IMG_OUT_W, IMG_OUT_H))
    
    # get corners
    y_min = target[0]
    x_min = target[1]
    y_max = target[2]
    x_max = target[3]
    
    # convert from corner coordinates to x_center, y_center, width, height
    y_center, x_center = (y_min + y_max)/2., (x_min + x_max)/2.
    bbox_h, bbox_w = y_max - y_min, x_max - x_min

    # normalize these values s.t. image goes from 0 to 1 (helps for arbitary size image size)
    y_center /= img_in_h
    x_center /= img_in_w
    bbox_h /= img_in_h
    bbox_w /= img_in_w
    
    class_idx = int(OBJECT_LABELS['hand'][0])
    target_normalized = np.array([y_center, x_center, bbox_h, bbox_w, class_idx], dtype=np.float32) 
    
    
    return img, target_normalized

def get_iou(hw1, hw2):
    # hw: (height, width)
    # assumption: both boxes have same centers
    
    # get extremes of both boxes
    hw1_max, hw2_max = hw1/2., hw2/2.
    hw1_min, hw2_min = -hw1_max, -hw2_max
    
    # get intersection area
    intersection_min = np.maximum(hw1_min, hw2_min)
    intersection_max = np.minimum(hw1_max, hw2_max)
    hw_intersection = np.maximum(intersection_max-intersection_min, 0.)
    area_intersection = hw_intersection[0] * hw_intersection[1]
    
    # get union area
    area_hw1 = hw1[0] * hw1[1]
    area_hw2 = hw2[0] * hw2[1]
    area_union = area_hw1 + area_hw2 - area_intersection
    
    iou = area_intersection / area_union
    
    return iou

def target2label(target):
    # initialize return data
    label = np.zeros((GRID_H, GRID_W, NUM_ANCHORS, 6), dtype=np.float32)  # 6: [offset_y, offset_x, scale_h, scale_w, class_idx, prob_obj]
    
    target_class = target[4]

    # map bbox from [0,1] space to [0,19] space
    bbox = target[0:4] * np.array([GRID_H, GRID_W, GRID_H, GRID_W])

    # get grid index for bbox center
    idx_y = int(floor(bbox[0]))
    idx_x = int(floor(bbox[1]))
    
    # find best anchor corresponding to bbox
    iou_best, idx_anchor_best = 0., 0
    for idx_anchor, anchor in enumerate(ANCHORS):
        iou = get_iou(bbox[2:4], anchor)
        if iou > iou_best:
            iou_best = iou
            idx_anchor_best = idx_anchor

    # update label
    if iou_best > 0.:
        label[idx_y, idx_x, idx_anchor_best] = np.array(
            [
                bbox[0] - idx_y,  # offset of box_center from top-left corner of grid containing box_center
                bbox[1] - idx_x,
                bbox[2]/ANCHORS[idx_anchor_best,0], # scale of anchor box so as to fit the bbox
                bbox[3]/ANCHORS[idx_anchor_best,1],
                target_class,
                1.0  # prob_object (object is present with prob=1)
            ], dtype=np.float32
        )
    return label
        
def get_processed_data(img, target):
    # normalize input and output
    img, target = normalize_data(img, target)
    # target.shape = (1, 5)
    # 5 corresponds to (c_y, c_x, h, w, class_idx)
    
    label = target2label(target)
    
    return img, label

## Write data to TFRecord format

In [5]:
# conversion functions (data to feature data types)
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def write_example_to_TFRecord(img, target, writer):
    # get processed data
    img, label = get_processed_data(img, target)
    
    # create example from this data
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'img': _bytes_feature(img.tostring()),
                'label': _bytes_feature(label.tostring()) 
            }
        )
    )

    writer.write(example.SerializeToString())

# for each camera in each data sequence
def write_data_to_TFRecord(img_paths, target_paths, out_dir):
    # write data into multiple TFRecord files
    idx_tfrecord, idx_data = 0, 0
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    while idx_data < len(img_paths):
        # new TFRecord file
        filename_tfrecord = os.path.join(out_dir, str(idx_tfrecord) + '.tfrecords')
        with tf.python_io.TFRecordWriter(filename_tfrecord) as writer:
            # write examples into this file until limit is reached
            idx_example = 0
            while idx_data < len(img_paths) and idx_example < NUM_EXAMPLES_PER_TFRECORD:
                path = img_paths[idx_data]
                img = cv2.imread(path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                target = np.loadtxt(target_paths[idx_data], usecols=1)
                idx_data += 1
                
                write_example_to_TFRecord(img, target, writer)
                idx_example += 1
            idx_tfrecord += 1

In [6]:
data_dirs = natsorted(os.listdir(DIR_INPUT))
for data_dir in data_dirs:
    for idx_cam in range(4):  # for 4 views
        input_path = os.path.join(DIR_INPUT, data_dir)
        img_names = os.listdir(input_path)
        pattern = 'webcam_'+str(idx_cam+1)
        img_names = natsorted(list(filter(lambda name: pattern in name, img_names)))
        img_paths = [os.path.join(input_path, name) for name in img_names]
        
        label_path = os.path.join(DIR_OUTPUT, data_dir)
        label_names = os.listdir(label_path)
        pattern = 'bbox_'+str(idx_cam+1)
        label_names = natsorted(list(filter(lambda name: pattern in name, label_names)))
        label_paths = [os.path.join(label_path, name) for name in label_names]
        
        out_dir = os.path.join(DIR_TFRECORDS, data_dir)
        out_dir = os.path.join(out_dir, str(idx_cam))
    
        write_data_to_TFRecord(img_paths, label_paths, out_dir)
        
        print('Finished', data_dir, idx_cam)

IndexError: index 13 is out of bounds for axis 1 with size 13